In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - AutoGluon

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_autogluon.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_autogluon.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates finetuning a PyTorch based [AutoGluon model for tabular data](https://auto.gluon.ai/stable/tutorials/tabular/index.html) on CPU and deploying it on Vertex AI for online prediction.

### Objective

In this tutorial, you learn how to:

- Finetune a PyTorch AutoGluon tabular model.
- Upload the model to [Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model on [Endpoint](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for tabular data.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Training
- Vertex AI Model Registry
- Vertex AI Online Prediction

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin


In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 3. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

import datetime
import importlib
import os
import uuid
from typing import Tuple

from google.cloud import aiplatform

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

models, endpoints = {}, {}

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "autogluon")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

## Finetune AutoGluon models

In [ ]:
# @title Finetune

# @markdown Create and run the training job with the model-garden PyTorch AutoGluon training docker using the Vertex AI SDK. This step takes around 15 minutes to finish.
# Set up training docker arguments.

job_name = common_util.get_job_name_with_datetime(prefix="pytorch-autogluon")

finetuning_workdir = os.path.join(BUCKET_URI, job_name)

# @markdown `train_data_path` - The path to the training data. It can also be a GCS path.
train_data_path = "https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/train.csv"  # @param {type:"string"}

# @markdown `label` - The column id to predict.
label = "signature"  # @param {type:"string"}

# The pre-built training docker image.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-autogluon-train:20240917_0827_RC00"

docker_args_list = [
    "--train_data_path",
    common_util.gcs_fuse_path(train_data_path),
    "--label",
    label,
    "--model_save_path",
    common_util.gcs_fuse_path(finetuning_workdir),
]
print("Running training job with args:")
print(" \\\n".join(docker_args_list))

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
    is_for_training=True,
)


# Add labels for the finetuning job.
labels = {
    "mg-source": "notebook",
    "mg-notebook-name": "model_garden_pytorch_autogluon.ipynb".split(".")[0],
}

labels["mg-tune"] = "publishers-autogluon-models-autogluon"
versioned_model_id = "pytorch-autogluon"
labels["versioned-mg-tune"] = f"{labels['mg-tune']}-{versioned_model_id}"

# Create and run the training job.
# Click on the generated link in the output under "View backing custom job:" to see your run in the Cloud Console.
job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
    labels=labels,
)
job.run(
    args=docker_args_list,
    base_output_dir=f"{finetuning_workdir}",
    replica_count=1,
    machine_type="g2-standard-12",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

## Deploy AutoGluon models

In [ ]:
# @title Deploy
# @markdown Upload the trained model and deploy it to an endpoint for prediction. This step takes around 15 minutes to finish.

# The pre-built serving docker image.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-autogluon-serve:20240924_0738_RC00"


# Deploy the trained model to Vertex AI Endpoint for prediction.
def deploy_model(
    model_path, model_name, machine_type
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploy the model to Vertex AI Endpoint for prediction."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_path,
        "DEPLOY_SOURCE": "notebook",
    }
    # Since the model_id is a GCS path, use artifact_uri to pass it
    # to the serving docker.
    artifact_uri = model_path
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[8080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
        model_garden_source_model_name="publishers/autogluon-ai/models/autogluon",
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
        system_labels={"NOTEBOOK_NAME": "model_garden_pytorch_autogluon.ipynb"},
    )
    return model, endpoint


models["model"], endpoints["endpoint"] = deploy_model(
    model_path=finetuning_workdir,
    model_name=common_util.get_job_name_with_datetime(prefix="autogluon"),
    machine_type="n1-highmem-16",
)

In [ ]:
# @title Prediction

Unnamed = 70746  # @param{type:"integer"}
ChernSimons = 0.0905302166938781  # @param {type:"number"}
CuspVolume = 12.226321765565215  # @param {type:"number"}
HyperbolicAdjointTorsionDegree = 0  # @param {type:"number"}
HyperbolicTorsionDegree = 10  # @param {type:"number"}
InjectivityRadius = 0.5077560544013977  # @param {type:"number"}
LongitudinalTranslation = 10.685555458068848  # @param {type:"number"}
MeridinalTranslation = 1.1441915035247805  # @param {type:"number"}
MeridinalTranslationReal = -0.5191566348075867  # @param {type:"number"}
ShortGeodesicImag = -2.7606005668640137  # @param {type:"number"}
ShortGeodesicReal = 1.0155121088027954  # @param {type:"number"}
Symmetry = 0.0  # @param {type:"number"}
SymmetryD3 = 0.0  # @param {type:"number"}
SymmetryD4 = 0.0  # @param {type:"number"}
SymmetryD6 = 0.0  # @param {type:"number"}
SymmetryD8 = 0.0  # @param {type:"number"}
SymmetryZ = 1.0  # @param {type:"number"}
Volume = 11.393224716186523  # @param {type:"number"}
instances = [
    {
        "Unnamed: 0": 70746,
        "chern_simons": 0.0905302166938781,
        "cusp_volume": 12.226321765565215,
        "hyperbolic_adjoint_torsion_degree": 0,
        "hyperbolic_torsion_degree": 10,
        "injectivity_radius": 0.5077560544013977,
        "longitudinal_translation": 10.685555458068848,
        "meridinal_translation_imag": 1.1441915035247805,
        "meridinal_translation_real": -0.5191566348075867,
        "short_geodesic_imag_part": -2.7606005668640137,
        "short_geodesic_real_part": 1.0155121088027954,
        "Symmetry_0": 0.0,
        "Symmetry_D3": 0.0,
        "Symmetry_D4": 0.0,
        "Symmetry_D6": 0.0,
        "Symmetry_D8": 0.0,
        "Symmetry_Z/2 + Z/2": 1.0,
        "volume": 11.393224716186523,
    },
]
predictions = endpoints["endpoint"].predict(instances=instances).predictions
print(predictions)

## Clean up resources

In [ ]:
# @title Delete the models and endpoints
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME